# Retrieval Augmented Generation

In [ ]:
!pip install -q langchain langchain_aws langchain_openai dotenv

In [ ]:
from langchain_aws import ChatBedrock
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
load_dotenv()

llm = ChatOpenAI(
    model="meta-llama/Llama-3.1-8B-Instruct",  
    openai_api_base=f"http://{os.getenv('MODEL_IP')}:8000/v1",
    openai_api_key="EMPTY"  # vLLM doesn’t require real keys
)


In [ ]:
ai_msg = llm.invoke("Which VLANs are reverved in my network?")
print(ai_msg.content)


## Ingestion Pipeline (Add data to Vector DB)
### Load

In [ ]:
from langchain.document_loaders import TextLoader

loader = TextLoader("acme_policies.md")
documents = loader.load()


### Split

In [ ]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
md_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

In [ ]:
md_splits = md_splitter.split_text(documents[0].page_content)
md_splits[5].metadata

In [ ]:
md_splits[5].page_content

In [ ]:
len(md_splits)

### Embedd

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_aws import BedrockEmbeddings

embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v2:0",
    region_name='us-east-1'
)

vector_store = InMemoryVectorStore(embeddings)
vector_store.add_documents(documents=md_splits)

In [ ]:
for index, (id, doc) in enumerate(vector_store.store.items()):
    if index < 2:
        print(doc['text'])
        print(f"{id}: {doc['vector'][0:5]} ({len(doc['vector'])})")
        print('----------------------------------------------------------------------------------------------------')

### Similarity Search

In [ ]:
query = "approved security mechanisms"
results = vector_store.similarity_search(query, k=2)
for result in results:
    print(result.page_content)

### RAG Pipeline (Retrieve context from Vector DB)

In [ ]:
from langchain import hub
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate

retrieval_qa_chat_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a network engineer assistant. Use the following documentation to answer the question.'
If the answer is not in the documentation, say you don’t know.

Documentation:
{context}

Question: {question}

Answer:
"""
)

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
    
rag_chain = (
    {'context': vector_store.as_retriever() | format_docs, 'question' : RunnablePassthrough()} |
    retrieval_qa_chat_prompt |
    llm
)
result = rag_chain.invoke("Which VLANs are reverved in the ACME network?")
print(result.content)